In [90]:
import binascii
def xlua_crc32(data):
    #crc = 0
    #for c in data:
    #    val = xlua_crcTable[(crc & 0xff) ^ c]
    #    crc = val ^ (crc >> 8) & 0xffffffff
    #return crc
    return ~binascii.crc32(data, 0xffffffff) & 0xffffffff

assert xlua_crc32(b'XPackage') == 0xa7a6904

In [116]:
xlua_crc32(b'@Config@.LuaCfg.DialogueCfg.PhoneCall_FY')

3239613863

In [91]:
def xlua_dec(data, key):
    kk = 0
    for c in key:
        kk ^= c
    return bytes([c ^ kk for c in data])

In [92]:
import io
import struct
class DataTransfer():
    def __init__(self, f):
        self.f = f
    
    def read(self, n):
        ret = self.f.read(n)
        if len(ret) != n:
            raise Exception("short read")
        return ret
    
    def readUInt16(self):
        return int.from_bytes(self.read(2), 'little')
    def readUInt32(self):
        return int.from_bytes(self.read(4), 'little')
    def readFloat(self):
        return struct.unpack("f", self.read(4))[0]
    def readString(self):
        return self.read(self.readUInt16()).decode()
    def readList(self, readFunc):
        listLen = self.readUInt32()
        ret = []
        for i in range(listLen):
            ret.append(readFunc(self))
        return ret
    def readUInt32List(self):
        return self.readList(lambda t: t.readUInt32())


In [93]:
ASSETBASE = '/mnt/d/Workspaces/IDAWorkspace/evol_deepspace/lysk/assets'

In [94]:
# /mnt/d/Workspaces/IDAWorkspace/evol_deepspace/lysk/assets
import io
def loadXFileZip(assetBasePath):
    xpackagePath = assetBasePath + f"/XFileZip/{xlua_crc32(b'XFileZip')}.bin"
    key = b'x3luapackage.XPackage'
    with open(xpackagePath, 'rb') as f:
        data = xlua_dec(f.read(), key)
    stream = DataTransfer(io.BytesIO(data))
    ret = {}
    mapLen = stream.readUInt16()
    ret['m_Map'] = {stream.readUInt32(): stream.readUInt32() for i in range(mapLen)}
    localeLen = stream.readUInt16()
    ret['m_LocaleMap'] = {stream.readUInt32(): stream.readString() for i in range(localeLen)}
    return ret

s_XFileHashConf = loadXFileZip(ASSETBASE)
s_XFileHashConf

{'m_Map': {198282928: 1847559878,
  242852497: 1852137922,
  273217593: 2897144768,
  352912883: 2812080556,
  421144121: 2396432823,
  511173152: 1381934351,
  632562875: 3197581174,
  794457740: 2042499512,
  968647023: 102887830,
  1146882285: 1886243960,
  1377838215: 2416692904,
  1486831245: 542482373,
  1512766901: 1299086652,
  1582219589: 4225495948,
  1769001654: 1866128195,
  1786119930: 170597155,
  1810957322: 1547285772,
  1844150263: 614380168,
  1847400111: 2045198964,
  1850269039: 3253515629,
  1917940458: 1365088183,
  1965720562: 3457849534,
  2148759427: 3784785347,
  2273227674: 827757865,
  2992418525: 1289357803,
  3017518125: 1284311731,
  3201489291: 2143591928,
  3256136950: 2044712363,
  3468480695: 1786200843,
  3476804167: 2806449518,
  3486057994: 3873922689,
  3526590587: 2239426227,
  3551822475: 324391429,
  3656241711: 2988537051,
  3736293960: 3483662229,
  3752476996: 331559878,
  3826703961: 287918816,
  3851541673: 2703386524,
  3852461016: 309454

In [111]:
import os
import json
os.makedirs(f'{ASSETBASE}/XPackage_dec/', exist_ok=True)
with open(f'{ASSETBASE}/XPackage_dec/{xlua_crc32(b"XFileZip")}_XFileZip_XFileHashConf.json', 'w') as f:
    json.dump(s_XFileHashConf, f, indent=4)

In [96]:
import zipfile
import lzma
import os

def extractAllXFileZip(zipPath, outDir):
    with zipfile.ZipFile(zipPath, 'r') as z:
        for name in z.namelist():
            assert name.endswith('.nx') or name.endswith('.nxf')
            with z.open(name) as f:
                f._expected_crc = None
                compressed = f._read2(0xffffffff)
            data = lzma.decompress(compressed)
            #key = b'x3luapkgencode.nx.nxf'
            #decdata = xlua_dec(data, key)
            
            with open(f'{outDir}/{name}', 'wb') as f:
                f.write(data)

XFileZipDir = f'{ASSETBASE}/XFileZip'
XPackageDir = f'{ASSETBASE}/XPackage'
for zipFile in os.listdir(XFileZipDir):
    if not zipFile.endswith('.zip'):
        continue
    extractAllXFileZip(f'{XFileZipDir}/{zipFile}', XPackageDir)

In [97]:
# /mnt/d/Workspaces/IDAWorkspace/evol_deepspace/lysk/assets
import io
def loadXPackage(assetBasePath):
    xpackagePath = assetBasePath + f"/XPackage/{xlua_crc32(b'XPackage')}.nx"
    print(xpackagePath)
    key = b'x3luapackage.XPackage'
    with open(xpackagePath, 'rb') as f:
        data = xlua_dec(f.read(), key)
    stream = DataTransfer(io.BytesIO(data))
    ret = {}
    ret['Encode'] = stream.readString()
    ret['LiveDt'] = stream.readUInt16()
    ret['PackageFileConfId'] = stream.readUInt32()
    ret['PackageConfId'] = stream.readUInt32()
    
    ret['PackageExt'] = stream.readString()
    ret['PackageConfExt'] = stream.readString()
    pathCount = stream.readUInt16()
    ret['UITextPaths'] = [stream.readString() for _ in range(pathCount)]
    return ret

s_XFileBaseConf = loadXPackage(ASSETBASE)
s_XFileBaseConf

/mnt/d/Workspaces/IDAWorkspace/evol_deepspace/lysk/assets/XPackage/175794436.nx


{'Encode': 'x3luapkgencode.nx.nxf',
 'LiveDt': 60,
 'PackageFileConfId': 587645444,
 'PackageConfId': 4285613305,
 'PackageExt': '.nx',
 'PackageConfExt': '.nxf',
 'UITextPaths': ['Locale.zh-CN.LuaCfg.UITextData',
  'Locale.zh-TW.LuaCfg.UITextData',
  'Locale.en-US.LuaCfg.UITextData',
  'Locale.ja-JP.LuaCfg.UITextData',
  'Locale.ko-KR.LuaCfg.UITextData']}

In [112]:
os.makedirs(f'{ASSETBASE}/XPackage_dec/', exist_ok=True)
with open(f'{ASSETBASE}/XPackage_dec/{xlua_crc32(b"XPackage")}_XPackage_XFileBaseConf.json', 'w') as f:
    json.dump(s_XFileBaseConf, f, indent=4)

In [107]:
import io
def loadPackageFileConf(assetBasePath, packageFileConfId, key):
    xpackagePath = assetBasePath + f"/XPackage/{packageFileConfId}.nx"
    print(xpackagePath)
    #key = b'x3luapkgencode.nx.nxf'
    with open(xpackagePath, 'rb') as f:
        data = xlua_dec(f.read(), key)
    stream = DataTransfer(io.BytesIO(data))
    count = int(len(data) / 6)
    ret = {}
    ret['m_Map'] = {stream.readUInt32(): stream.readUInt16() for i in range(count)}
    return ret

s_XFileConf = loadPackageFileConf(ASSETBASE, s_XFileBaseConf['PackageFileConfId'], s_XFileBaseConf['Encode'].encode())
s_XFileConf

/mnt/d/Workspaces/IDAWorkspace/evol_deepspace/lysk/assets/XPackage/587645444.nx


{'m_Map': {251123: 3,
  673025: 38,
  2781320: 2,
  2810421: 10,
  2894448: 10,
  3655782: 18,
  4956318: 38,
  5959940: 17,
  6687955: 2,
  7932305: 28,
  8091988: 38,
  8105741: 2,
  8203502: 2,
  11108466: 3,
  11112718: 10,
  12320176: 3,
  12492125: 22,
  12946859: 3,
  13159109: 37,
  14030947: 26,
  14391100: 2,
  14742543: 38,
  15393562: 0,
  15465103: 21,
  15700872: 2,
  16430913: 3,
  16522477: 30,
  17334434: 2,
  17542702: 2,
  18348579: 23,
  18680856: 2,
  19132903: 3,
  19752494: 0,
  20465207: 14,
  20839049: 1,
  21914279: 10,
  22078618: 3,
  22934529: 17,
  23605927: 3,
  24034818: 10,
  24425283: 2,
  24626875: 2,
  26255395: 0,
  26542891: 5,
  28009245: 2,
  28370627: 0,
  31006191: 2,
  33483613: 2,
  34227127: 0,
  34496916: 2,
  34826364: 10,
  35115239: 0,
  36780487: 26,
  37402352: 2,
  38668956: 10,
  40783550: 3,
  41448854: 28,
  41538275: 3,
  41843611: 34,
  42112947: 16,
  42218966: 2,
  43550379: 3,
  43793039: 2,
  44953079: 0,
  45396065: 3,
  456

In [113]:
os.makedirs(f'{ASSETBASE}/XPackage_dec/', exist_ok=True)
with open(f'{ASSETBASE}/XPackage_dec/{s_XFileBaseConf["PackageFileConfId"]}_FileConf_XFileConf.json', 'w') as f:
    json.dump(s_XFileConf, f, indent=4)

In [101]:
# /mnt/d/Workspaces/IDAWorkspace/evol_deepspace/lysk/assets
import io
def loadPackageConf(assetBasePath, packageConfId, key):
    xpackagePath = assetBasePath + f"/XPackage/{packageConfId}.nx"
    print(xpackagePath)
    #key = b'x3luapkgencode.nx.nxf'
    with open(xpackagePath, 'rb') as f:
        data = xlua_dec(f.read(), key)
    stream = DataTransfer(io.BytesIO(data))
    count = int(len(data) / 6)
    ret = {}
    ret['m_Map'] = {stream.readUInt16(): stream.readUInt32() for i in range(count)}
    return ret

s_XFilePackageConf = loadPackageFileConf(ASSETBASE, s_XFileBaseConf['PackageConfId'], s_XFileBaseConf['Encode'].encode())
s_XFilePackageConf

/mnt/d/Workspaces/IDAWorkspace/evol_deepspace/lysk/assets/XPackage/4285613305.nx


{'m_Map': {0: 198282928,
  1: 242852497,
  2: 273217593,
  3: 352912883,
  4: 421144121,
  5: 511173152,
  6: 632562875,
  7: 968647023,
  8: 1146882285,
  9: 1377838215,
  10: 1486831245,
  11: 1512766901,
  12: 1582219589,
  13: 1769001654,
  14: 1786119930,
  15: 1810957322,
  16: 1844150263,
  17: 1847400111,
  18: 2148759427,
  19: 2273227674,
  20: 2992418525,
  21: 3017518125,
  22: 3256136950,
  23: 3468480695,
  24: 3476804167,
  25: 3486057994,
  26: 3526590587,
  27: 3551822475,
  28: 3656241711,
  29: 3736293960,
  30: 3752476996,
  31: 3826703961,
  32: 3851541673,
  33: 3852461016,
  34: 3985220460,
  35: 4034495244,
  36: 4184666876,
  37: 4266635223,
  38: 4269019671}}

In [114]:
os.makedirs(f'{ASSETBASE}/XPackage_dec/', exist_ok=True)
with open(f"{ASSETBASE}/XPackage_dec/{s_XFileBaseConf['PackageConfId']}_FilePackageConf_XFilePackageConf.json", 'w') as f:
    json.dump(s_XFilePackageConf, f, indent=4)

In [64]:
def packageExists(assetBasePath, packageId):
    packagesDir = f'{assetBasePath}/XPackage'
    nxFile = f'{packagesDir}/{packageId}.nx'
    return os.path.exists(nxFile)

def splitXLuaNxf(assetBasePath, packageId, key, fileExt, outDir):
    packagesDir = f'{assetBasePath}/XPackage'
    nxfFile = f'{packagesDir}/{packageId}.nxf'
    nxFile = f'{packagesDir}/{packageId}.nx'
    assert os.path.exists(nxfFile)

    os.makedirs(outDir, exist_ok=True)

    with open(nxfFile, 'rb') as f:
        nxfData = xlua_dec(f.read(), key)
    nxfMap = {}
    for off in range(0, len(nxfData), 8):
        idx = int.from_bytes(nxfData[off:off+4], 'little')
        off = int.from_bytes(nxfData[off+4:off+8], 'little')
        nxfMap[idx] = off
    with open(nxFile, 'rb') as f:
        data = xlua_dec(f.read(), key)
    for idx, off in nxfMap.items():
        with open(f'{outDir}/{idx}.{fileExt}', 'wb') as w:
            flen = int.from_bytes(data[off:off+4], 'little')
            w.write(data[off+4:off+4+flen])

            
allpackages = set(
    list(s_XFilePackageConf['m_Map'].values()) + # index -> id
    list(s_XFileHashConf['m_Map'].keys()) + # id -> hash
    list(s_XFileHashConf['m_LocaleMap'].keys())) # id -> locale path
for packageId in allpackages:
    print(f"Extracting {packageId}")
    if not packageExists(ASSETBASE, packageId):
        print("package %s not exists" % packageId)
        continue
    if packageId in s_XFileHashConf['m_LocaleMap']:
        localePath = s_XFileHashConf['m_LocaleMap'][packageId]
        outDir = f'{ASSETBASE}/XPackage_dec/{localePath}'
        fileExt = 'txt'
    else:
        outDir = f'{ASSETBASE}/XPackage_dec/Lua/{packageId}'
        fileExt = 'luac'
    splitXLuaNxf(ASSETBASE, packageId, s_XFileBaseConf['Encode'].encode(), fileExt, outDir)

Extracting 2148759427
Extracting 1377838215
Extracting 1810957322
package 1810957322 not exists
Extracting 3486057994
Extracting 3551822475
Extracting 1486831245
Extracting 4034495244
Extracting 794457740
package 794457740 not exists
Extracting 3201489291
Extracting 242852497
package 242852497 not exists
Extracting 4225581834
package 4225581834 not exists
Extracting 4269019671
Extracting 2273227674
Extracting 511173152
Extracting 3851541673
package 3851541673 not exists
Extracting 3017518125
Extracting 1847400111
Extracting 198282928
Extracting 3656241711
Extracting 1512766901
package 1512766901 not exists
Extracting 1769001654
Extracting 3468480695
Extracting 421144121
Extracting 273217593
Extracting 632562875
Extracting 3752476996
Extracting 1582219589
Extracting 3476804167
package 3476804167 not exists
Extracting 3736293960
Extracting 4266635223
Extracting 3852461016
Extracting 3826703961
Extracting 2992418525
package 2992418525 not exists
Extracting 1917940458
package 1917940458 no